# Homework 3  -  Roger Chow (r6chow)

In [1]:
import networkx as nx
import warnings
import math
import operator
warnings.filterwarnings("ignore", category=UserWarning)

## Data Prep: Load political blog dataset

In [2]:
#Read edge list into Directed graph
pol_DiG=nx.read_edgelist("polblogs.edgelist.simple_format_unweighted", 
                   nodetype=str,
                   data=(('weight',float),),
                   create_using=nx.DiGraph())

## (1) Implement Closeness Centrality from scratch (7)

The implementation of Closeness Centrality requires the identification of shortest path between nodes.  We will use Dijkstra implementation from Homework 2.

In [3]:
def Closeness_Centrality(G, N=None):
    
    closeness = {}
    
    for n in G.nodes():
    
        #get the shortest path from node N
        dist_dict = Dijkstra(G, n)

        sum_distance = 0

        #sum the n-1 distances reachable from N
        for node in dist_dict:
            if node != N:
                sum_distance += len(dist_dict[node])-1

        #closeness centrality (week 3 lecture) 
        # CLV(v) = [|V| - 1] / [SUM(distance(v, vi))]

        if sum_distance > 0 :
            closeness[n] = ((len(dist_dict) -1) / sum_distance )
        else:
            closeness[n] = 0
    
    if N is not None:
        return closeness[N]
    else:
        return closeness
    
    

# Return a dictionary of shortest path from Node N (Homework 2)
def Dijkstra(G, N):
    
    # for directed Graph, we need to reverse the view
    if G.is_directed():
        G = G.reverse()
        
    unvisited_nodes = list(G.nodes())
    visited_nodes = []
    distances = {}
    prev = {}

    #initialize distances of all nodes from N as infinity
    for n in unvisited_nodes:
        distances[n] = math.inf
        prev[n] = ''

    #initialize distance of node N as 0
    distances[N] = 0

    while (len(unvisited_nodes) > 0):

        #pick node X from unvisited array with shortest distance from N
        min_dist = min([distances[x] for x in unvisited_nodes])
        X = ''
        for x in unvisited_nodes:
            if (distances[x] <= min_dist):
                X = x

        #for each vertex Y that is a neighbour of X
        neighbors_X = [n for n in nx.neighbors(G, X)]
        
        for Y in neighbors_X:

            #compute the distance of Y from N using X as an intermediary node
            weight = 1  #constant for unweighted
            
            new_dist_Y = distances[X] + weight

            #if the new_distance(Y) < old_distance(Y) then replace old_distance(Y) with new_distance(Y) and prev_vertex of Y with X
            if (new_dist_Y < distances[Y]):
                distances[Y] = new_dist_Y
                prev[Y] = X

        visited_nodes.append(X)
        unvisited_nodes.remove(X)
   

    #shortest paths reachable
    shortest_path = {}
    for node in prev.keys():
        path = getPath(node, prev)
        if (path[0] == N):
            shortest_path[node] = getPath(node, prev)
    
    #sort and return the dictionary of shortest paths for node N
    return dict((x, y) for x, y in sorted(shortest_path.items(), key = lambda kv:len(kv[1])))


#helper function to get the path to node n based on prev 
def getPath(n, prev):
    if (prev[n] == ''):
        return ([n])
    else:
        return (getPath(prev[n], prev))+[n]



#### Compare implementation with Networkx implementation results

In [4]:
result = Closeness_Centrality(pol_DiG)

In [5]:
max(result.items(), key=operator.itemgetter(1))[0]

'lashawnbarber.com'

In [6]:
result['lashawnbarber.com']

1.0

In [7]:
result['instapundit.com']

0.5185185185185185

In [8]:
nx_result = nx.closeness_centrality(pol_DiG, wf_improved=False)

In [9]:
max(nx_result.items(), key=operator.itemgetter(1))[0]

'lashawnbarber.com'

In [10]:
nx_result['lashawnbarber.com']

1.0

In [11]:
nx_result['instapundit.com']

0.5185185185185185

## (2) Implement Page Rank (using Damping factor approach) from scratch (7)

In [12]:
def PageRank(G, d=0.85, tol=1e-05, max_iter=100):
    
    #initialize all page range to 1 / |V|
    G_size = len(G.nodes())
    curr_pagerank = dict.fromkeys(G.nodes(), 1/G_size)
    
    # convert to stochastic graph where the sum of out-edges adds to 1 
    # [shortcut for getting connecting nodes contribution]
    A = nx.stochastic_graph(G)

    # find all the dangling nodes (out degree is 0)
    nodes_dangling = [node for node in A if A.out_degree(node) == 0.0] 
    
    # iterate until max iterations reach or convergence reached
    for i in range(max_iter):
        prev_pagerank = curr_pagerank
        curr_pagerank = dict.fromkeys(prev_pagerank, 0)
        
        #sum up the dangling nodes
        sum_nodes_dangling = d * sum(prev_pagerank[node] for node in nodes_dangling) 
        
        #compute new page rank for each node  i.e.  PR(A)/C(A) + ... 
        for node in curr_pagerank:
            
            for conn_node in A[node]:
                curr_pagerank[conn_node] += d * prev_pagerank[node] * A[node][conn_node]['weight']
                
            #based on http://home.ie.cuhk.edu.hk/~wkshum/papers/pagerank.pdf, dangling node have probablity (1/n)
            curr_pagerank[node] += (sum_nodes_dangling / G_size) +  ((1.0 - d) / G_size)
        
        #check for convergence if the total difference between previous and current page rank is less than tolerance
        if sum([abs(curr_pagerank[node] - prev_pagerank[node]) for node in curr_pagerank]) < tol:
            return  curr_pagerank
        
    return curr_pagerank

#### Compare implementation with Networkx implementation results

In [13]:
pageRankResult = PageRank(pol_DiG)

In [14]:
max(pageRankResult.items(), key=operator.itemgetter(1))[0]

'instapundit.com'

In [15]:
pageRankResult['instapundit.com']

0.03157503121429926

In [16]:
pageRankResult['lashawnbarber.com']

0.0002737427388847285

In [17]:
nx_PageRankResult = nx.pagerank(pol_DiG)

In [18]:
max(nx_PageRankResult.items(), key=operator.itemgetter(1))[0]

'instapundit.com'

In [19]:
nx_PageRankResult['instapundit.com']

0.03158762626582744

In [20]:
nx_PageRankResult['lashawnbarber.com']

0.00027398331593247766

## (3) How do the important nodes using the two methods compare? Are the node rankings the same or different? (1)

<B>Answer:</B>  The important node found using Closeness Centralality is 'lashawnbarber.com'.  However, using Page Ranking with dampening factor of 0.85, the important node found was 'instapundit.com'.   If we compare how these two nodes ranked in other methods, 'instapundit.com' was still in the top 100 nodes importance (ranked 74 out of 1224) based on Closeness Centrality.  Conversly, the node 'lashawnbarber.com' was found to be at the bottom half of the page rankings (ranked 859 out of 1224).  



In [21]:
sorted_closeness = sorted(result.items(), key=operator.itemgetter(1), reverse=True)

In [22]:
sorted_closeness[0:40]

[('lashawnbarber.com', 1.0),
 ('leonards-digest.blogspot.com', 1.0),
 ('tomburka.com', 1.0),
 ('bakshi.us/redskunk', 1.0),
 ('abbadabbaduo.blogspot.com', 1.0),
 ('stevemacek.blogspot.com', 1.0),
 ('lefttimes.com', 1.0),
 ('myleftbrain.com', 1.0),
 ('btcnews.com/btcnews', 1.0),
 ('paulchasman.com', 1.0),
 ('parabasis.typepad.com', 1.0),
 ('irregulartimes.com/blogger.html', 1.0),
 ('donkey2004.blogspot.com', 1.0),
 ('bushlies.net/pages/10/index.htm', 1.0),
 ('one38.org', 1.0),
 ('alvintostig.typepad.com', 1.0),
 ('radikal_papi.pitas.com', 1.0),
 ('radgeek.com', 1.0),
 ('erikack.blogspot.com', 1.0),
 ('theblueview.blogspot.com', 1.0),
 ('digital-democrat.blogspot.com', 1.0),
 ('quimundus.squarespace.com', 1.0),
 ('bullmooseblog.com', 1.0),
 ('thunewatch.squarespace.com', 1.0),
 ('simplyappalling.blogspot.com', 1.0),
 ('imprescindibile.ilcannocchiale.it', 1.0),
 ('ergio.blogspot.com', 1.0),
 ('enemykombatant.blogspot.com', 1.0),
 ('massachusetts-liberal.com', 1.0),
 ('grassrootsnation.com'

In [23]:
sorted_pagerank = sorted(pageRankResult.items(), key=operator.itemgetter(1), reverse=True)

In [24]:
sorted_pagerank[0:40]

[('instapundit.com', 0.03157503121429926),
 ('dailykos.com', 0.022109920485296256),
 ('right-thinking.com', 0.0162200808370096),
 ('rogerailes.blogspot.com', 0.014850212878956139),
 ('liberaloasis.com', 0.013919358539898674),
 ('michaelberube.com', 0.013197519511849318),
 ('prospect.org/weblog', 0.0124558513098053),
 ('denbeste.nu', 0.012119917396663644),
 ('kensain.com', 0.011963521496989751),
 ('mudvillegazette.com', 0.010940642821213415),
 ('dissectleft.blogspot.com', 0.010601908047248202),
 ('higherpieproductions.com', 0.010125738346325804),
 ('ladymac.com', 0.009925243300367417),
 ('blogsforbush.com', 0.009719367275369296),
 ('dolphinsdock.com/blog', 0.009589959267518015),
 ('pardonmyenglish.com', 0.009385947292756736),
 ('thepoorman.net', 0.009369571783782818),
 ('discourse.net', 0.008534771091938403),
 ('atrios.blogspot.com', 0.00812004834050041),
 ('writingcompany.blogs.com/this_isnt_writing_its_typ', 0.007735899484971417),
 ('counterspin.blogspot.com', 0.006848708925101928),
 

In [25]:
#find position of 'lashawnbarber.com' in pagerank
size = len(sorted_pagerank)
for i in range(len(sorted_pagerank)):
    if (sorted_pagerank[i][0] == 'lashawnbarber.com'):
        print('lashawnbarber.com is at position {0} out of {1}'.format(i+1, size))
    if (sorted_pagerank[i][0] == 'instapundit.com'):
        print('instapundit.com is at position {0} out of {1}'.format(i+1, size))
    

instapundit.com is at position 1 out of 1224
lashawnbarber.com is at position 859 out of 1224


In [26]:
#find position of 'lashawnbarber.com' in closeness
size = len(sorted_closeness)
for i in range(len(sorted_closeness)):
    if (sorted_closeness[i][0] == 'lashawnbarber.com'):
        print('lashawnbarber.com is at position {0} out of {1}'.format(i+1, size))
    if (sorted_closeness[i][0] == 'instapundit.com'):
        print('instapundit.com is at position {0} out of {1}'.format(i+1, size))
    

lashawnbarber.com is at position 1 out of 1224
instapundit.com is at position 74 out of 1224
